In [2]:
import pandas as pd
import random

In [3]:
df = pd.read_csv('/hpc/group/codeplus22-vis/infousa/derived set/2020/Household_Ethnicity_zip_49309_year_2020.txt', sep = '\t')
df = df[['ZIP', 'census_county_2010', 'STATE', 'ChildrenHHCount', 'length_of_residence', 
         'children_ind', 'head_hh_age_code', 'GE_LATITUDE_2010', 'GE_LONGITUDE_2010']]
df

,ZIP,census_county_2010,STATE,ChildrenHHCount,length_of_residence,children_ind,head_hh_age_code,GE_LATITUDE_2010,GE_LONGITUDE_2010
0,49309,123,MI,0,8,0,L,43.813295,-85.821729
1,49309,123,MI,0,15,0,I,43.782101,-85.785538
2,49309,123,MI,3,5,1,K,43.799797,-85.826307
3,49309,123,MI,0,27,0,M,43.736318,-85.798551
4,49309,123,MI,0,27,0,M,43.808913,-85.832011
...,...,...,...,...,...,...,...,...,...
1263,49309,123,MI,0,1,0,E,43.797361,-85.848833
1264,49309,123,MI,0,1,0,F,43.771055,-85.809882
1265,49309,123,MI,0,1,0,K,43.713169,-85.866478
1266,49309,123,MI,0,1,0,H,43.786915,-85.845480


In [5]:
%%time
ZIP = []
census_county = []
ChildrenHHCount = []
length_of_residence = []
children_ind = []
GE_LONGITUDE_2010 = []
GE_LATITUDE_2010 = []
STATE = []
head_hh_age_code = []

states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
age_codes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M']

for i in range (0, 10000):
    ZIP.append(random.randint(10000, 99999))
    census_county.append(random.randint(0, 999))
    STATE.append(states[random.randint(0, 50)])
    ChildrenHHCount.append(random.randint(0, 15))
    length_of_residence.append(random.randint(0, 70))
    children_ind.append(random.randint(0,1))
    head_hh_age_code.append(age_codes[random.randint(0, 12)])
    GE_LONGITUDE_2010.append(random.uniform(-125, -65))
    GE_LATITUDE_2010.append(random.uniform(35, 50))
    
print(len(ZIP))
print(len(census_county))
print(len(ChildrenHHCount))
print(len(length_of_residence))
print(len(children_ind))
print(len(GE_LONGITUDE_2010))
print(len(GE_LATITUDE_2010))
print(len(children_ind))

10000
10000
10000
10000
10000
10000
10000
10000
CPU times: user 79.7 ms, sys: 723 µs, total: 80.4 ms
Wall time: 79.6 ms


In [9]:
d = {'ZIP': ZIP, 'census_county': census_county, 'STATE': STATE, 'ChildrenHHCount': ChildrenHHCount, 
     'length_of_residence': length_of_residence, 'children_ind': children_ind, 'head_hh_age_code': head_hh_age_code, 
     'GE_LATITUDE_2010': GE_LATITUDE_2010, 'GE_LONGITUDE_2010': GE_LONGITUDE_2010}
df_synthetic = pd.DataFrame(d)
df_synthetic

,ZIP,census_county,STATE,ChildrenHHCount,length_of_residence,children_ind,head_hh_age_code,GE_LATITUDE_2010,GE_LONGITUDE_2010
0,26259,365,IA,1,2,0,D,42.385417,-75.315531
1,91480,133,MI,0,48,0,M,49.545403,-110.453664
2,32785,575,TX,4,56,1,E,48.787938,-99.605689
3,99856,620,NH,1,12,1,I,36.814040,-121.126992
4,53966,349,NM,9,62,0,D,43.098412,-95.737069
...,...,...,...,...,...,...,...,...,...
9995,51778,480,HI,12,8,1,E,42.043213,-117.237283
9996,57970,489,TX,13,63,0,D,49.914162,-77.157357
9997,17870,541,WA,1,37,0,C,40.977161,-120.444129
9998,43871,409,NY,2,17,1,D,48.631953,-88.827971


In [19]:
%%time
filepath = '/hpc/group/codeplus22-vis/synthetic_data/synthetic_infousa_test/Household_Ethnicity_zip_'

states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
age_codes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M']

for i in range(2):  
    zipcode = random.randint(10000, 99999)
    ZIP = []
    census_county = []
    ChildrenHHCount = []
    length_of_residence = []
    children_ind = []
    GE_LONGITUDE_2010 = []
    GE_LATITUDE_2010 = []
    STATE = []
    head_hh_age_code = []
    
    for i in range (0, 10000):
        ZIP.append(random.randint(10000, 99999))
        census_county.append(random.randint(0, 999))
        STATE.append(states[random.randint(0, 50)])
        ChildrenHHCount.append(random.randint(0, 15))
        length_of_residence.append(random.randint(0, 70))
        children_ind.append(random.randint(0,1))
        head_hh_age_code.append(age_codes[random.randint(0, 12)])
        GE_LONGITUDE_2010.append(random.uniform(-125, -65))
        GE_LATITUDE_2010.append(random.uniform(35, 50))
        
    d = {'ZIP': ZIP, 'census_county': census_county, 'STATE': STATE, 'ChildrenHHCount': ChildrenHHCount, 
     'length_of_residence': length_of_residence, 'children_ind': children_ind, 'head_hh_age_code': head_hh_age_code, 
     'GE_LATITUDE_2010': GE_LATITUDE_2010, 'GE_LONGITUDE_2010': GE_LONGITUDE_2010}
    df_synthetic = pd.DataFrame(d)
    df_synthetic.to_csv(filepath + str(zipcode) + '_year_2020_TEST.txt', sep = '\t', index = False)

CPU times: user 265 ms, sys: 7.36 ms, total: 272 ms
Wall time: 294 ms


In [20]:
f = pd.read_csv('/hpc/group/codeplus22-vis/synthetic_data/synthetic_infousa_test/Household_Ethnicity_zip_55471_year_2020_TEST.txt', sep = '\t')
f

,ZIP,census_county,STATE,ChildrenHHCount,length_of_residence,children_ind,head_hh_age_code,GE_LATITUDE_2010,GE_LONGITUDE_2010
0,78002,71,TX,4,67,1,G,38.783884,-85.164248
1,33553,872,DE,6,35,1,I,40.801337,-105.306761
2,87131,681,IA,7,65,1,M,46.264299,-111.298850
3,75488,409,CA,1,70,1,K,41.121055,-75.133467
4,17328,660,AL,0,49,0,I,35.240622,-116.736490
...,...,...,...,...,...,...,...,...,...
9995,35571,393,NY,2,66,1,I,40.552513,-123.182117
9996,52546,505,IL,5,4,1,L,37.332035,-115.747088
9997,38073,73,WI,7,68,1,B,40.303956,-94.569606
9998,69864,384,NC,14,35,1,H,37.074480,-101.211492
